# 滴滴比赛

导入必须的！

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### 文件位置

In [2]:

base_dir = "G:/project/dataset/citydata/season_1/training_data"

train_cluster_map_file = os.path.join(base_dir, 'cluster_map', 'cluster_map')
train_poi_file = os.path.join(base_dir, 'poi_data', 'poi_data')
train_order_file = os.path.join(base_dir, 'order_data', 'order_data_2016-01-21')
#train_cluster_map = os.path.join(base_dir, 'cluster_map')


## 区域 hashid 和 实际id

cluster_map 数据， pandas 应该比 dict 快， 到时候 merge 或 join 比 从 dict 里找好。

In [7]:
cluster_map = pd.read_csv(train_cluster_map_file, sep="\t", header=None, names=['hashid', 'id'])

In [13]:
print len(cluster_map)

66


In [65]:
cluster_map[:2]

,hashid,id
0,90c5a34f06ac86aee0fd70e2adce7d8a,1
1,f2c8c4bb99e6377d21de71275afd6cd2,2


## 复习下 pandas 读数据

1. 复习下数据读取, read_csv
2. 个人认为不需要的数据， 如 order_id 等id， 该省略的省掉（order_id, passenger_id, dest_district_has）， 该转换的转换, driver_id 变成是否有接单， 时间拆出日期和时间片， start_distrct_hash 改成实际id。 节省空间。去掉一堆 hash的字符串， 可以省很多空间

**也许有些信息还有用， 觉得有用时再补充回来**


In [61]:
# 时：分：秒 转成 10分钟的时间片
def time2Slice(time_str):
    l = time_str.split(":")
    return int(l[0]) * 6 + int(l[1]) / 10 + 1

In [62]:
# Order data 的数据 hash 太多， 顺便复习下 pandas
def reduceOrderFile(base_name, f):
    import time
    prev = time.time()
    fname = os.path.join(base_name, f)
    order = pd.read_csv(fname, sep="\t", header=None, dtype={'Price': np.str},
                          names=['order_id', 'driver_id', 'passenger_id', 'start_district_hash', 'dest_district_hash',
                                'Price', 'Time'])

    full_order = order.merge(cluster_map, left_on='start_district_hash', right_on='hashid')

    full_order['Date'] = pd.Series([e.split()[0] for e in full_order['Time']])

    full_order['TimePiece'] = pd.Series([time2Slice(e.split()[1]) for e in full_order['Time']])

    full_order['Resp'] = full_order['driver_id'].notnull()

    small_order = full_order.loc[:, ['Date', 'TimePiece', 'id', 'Price', 'Resp']]
    small_order_file = os.path.join(base_name, f + ".csv")
    small_order.to_csv(small_order_file, index = False)
    print time.time() - prev
    
    #return small_order

In [63]:
# 测试函数

def testReduceOrder():
    z = reduceOrderFile('G:/project/dataset/citydata/season_1/training_data\order_data', 'order_data_2016-01-21')
    print len(z)
    print sum(z['Resp'])

5.23000001907


In [64]:
# 批量 order reduce
def reduceFiles(base_name):
    for rt, dirs, files in os.walk(base_name):
        for f in files:           
            reduceOrderFile(base_name, f)


In [59]:
# 运行过就不要再来了， 一个文件6秒， 20个文件， 还是要2分钟的。
def runForOnce():
    reduceFiles(os.path.join(base_dir, 'order_data'))

6.16599988937
4.09300017357
4.24799990654
4.88099980354
5.09999990463
4.90000009537
4.94000005722
5.15599989891
5.15200018883
5.24800014496
6.0119998455
5.132999897
5.16599988937
5.06999993324
5.88199996948
6.375
4.98500013351
5.53900003433
6.41299986839
5.78500008583
5.19200015068
0.603999853134


In [43]:
def test_read_order():
    train_order = pd.read_csv(train_order_file, sep="\t", header=None, dtype={'Price': np.float, 'driver_id': np.str},
                          names=['order_id', 'driver_id', 'passenger_id', 'start_district_hash', 'dest_district_hash',
                                'Price', 'Time'])
    print len(train_order)
    t = train_order['driver_id']
    print t.notnull()
    small_order = full_train_order.loc[:, ['Date', 'TimePiece', 'id', 'Price', 'Resp']]

474340
